In [ ]:
BiocManager::install("DESeq2")
library(DESeq2)
library(dplyr)
library(tidyr)
library(ggplot2)
library(ggrepel)
library(pheatmap)
library(RColorBrewer)

In [ ]:
# CountData
count = read.csv("../input/mtle-hippo/mTLE_hippo.csv")

In [ ]:
# Data Cleaning
# Removing Low expreseed genes.
count = count%>%
mutate(RowSum = count%>%
       dplyr::select(-X)%>%
       rowSums())%>%
filter(RowSum>10)%>%
dplyr::select(-RowSum)
count$X = as.character(count$X)
count = as.data.frame(count,row.names = count$X)%>%select(-X)

In [ ]:
#ColData
Conditions = data.frame(row.names = colnames(count),
                       Cases = c(rep("Epilepsy",19),rep("Normal",4)))

In [ ]:
# Differential Test Analysis
all(rownames(Conditions) %in% colnames(count))
count <- count[, rownames(Conditions)]
all(rownames(Conditions) == colnames(count))
dds<-DESeqDataSetFromMatrix(count, colData = Conditions, design =~Cases)
dds<-DESeq(dds)
res<-results(dds)

In [ ]:
res = as.data.frame(res)
res = res[order(res$padj),]
head(res)

In [ ]:
#MA Plot
ma = res%>%
filter(padj!='NA' & log2FoldChange!='NA')%>%
select("baseMean","log2FoldChange","padj")
ma%>%ggplot(aes(x=log2(baseMean),y=log2FoldChange,color=ifelse((padj)<0.05,"Red","grey")))+
geom_point(color=ifelse((ma$padj)<0.05,ifelse((ma$log2FoldChange)>0,"#B31B21","#1465AC"),"darkgray"),size=2)+
geom_hline(yintercept=0,color = "red", size=1)+
geom_label_repel(data = head(ma%>%
                            filter(padj<0.05)%>%
                            filter(log2FoldChange>0)%>%
                            arrange(padj),5),
                aes(label = rownames(head(ma%>%
                                          filter(padj<0.05)%>%
                                          filter(log2FoldChange>0)%>%
                                          arrange(padj),5))),
                size = 3.5,
                box.padding = unit(0.35, "lines"),
                point.padding = unit(0.3, "lines"),
                color = "black",fontface = 'bold',
                hjust = 1,
                vjust = -1,
                fill = "#B31B21")+
geom_label_repel(data = head(ma%>%
                            filter(padj<0.05)%>%
                            filter(log2FoldChange<0)%>%
                            arrange(padj),5),
                aes(label = rownames(head(ma%>%
                                          filter(padj<0.05)%>%
                                          filter(log2FoldChange<0)%>%
                                          arrange(padj),5))),
                size = 3.5,
                box.padding = unit(0.35, "lines"),
                point.padding = unit(0.3, "lines"),
                color = "black",
                fontface = 'bold',
                hjust = -1,
                vjust = 2,
                fill = "#1465AC")+
ggtitle("MA Plot")+
labs(x="log2 Normalized mean",y="log2FC")+
theme_minimal()+
theme(plot.title = element_text(size=20, face="bold"),
      axis.title.x = element_text(size=15, face="bold"),
      axis.title.y = element_text(size=15, face="bold"),
      axis.text.x = element_text(size=13,face="bold"),
      axis.text.y = element_text(size=13,face="bold"))

In [ ]:
# Volcano Plot
vol = res%>%
filter(padj!='NA' & log2FoldChange!='NA')%>%
dplyr::select("log2FoldChange","padj")
vol%>%ggplot(aes(x = log2FoldChange , y = -log10(padj),color=ifelse((padj)<=0.05,"Red","black")))+
geom_point(color=ifelse((vol$padj)<=0.05,"#B31B21","darkgrey"))+
geom_label_repel(data = head(vol%>%
                             filter(padj<0.05)%>%
                             arrange(padj),10),
                aes(label = rownames(head(vol%>%
                                          filter(padj<0.05)%>%
                                          arrange(padj),10))),
                size = 3.5,
                box.padding = unit(0.35, "lines"),
                point.padding = unit(0.3, "lines"),
                color = "black",
                fontface = 'bold')+
ggtitle("Volcano Plot")+
theme_minimal()+
theme(plot.title = element_text(size=20, face="bold"),
      axis.title.x = element_text(size=15, face="bold"),
      axis.title.y = element_text(size=15, face="bold"),
      axis.text.x = element_text(size=13,face="bold"),
      axis.text.y = element_text(size=13,face="bold"))

In [ ]:
dds = collapseReplicates(dds,groupby = dds$Cases)
rld = rlog(dds)
head(assay(rld))

In [ ]:
# PCA PLot
pcaData = plotPCA(rld,intgroup="Cases", returnData = TRUE)
percentVar = round(100 * attr(pcaData, "percentVar"))
ggplot(pcaData, aes(PC1, PC2, color=factor(Cases), shape=factor(Cases)))+
geom_point(size = 4)+
geom_text(aes(label = name),vjust=1.7)+
xlab(paste0("PC1: ",percentVar[1],"% variance"))+
ylab(paste0("PC2: ",percentVar[2],"% variance"))+
xlim(-25,25)+
theme_minimal()+
theme(legend.title = element_blank())

In [ ]:
# Box-Plot
as.data.frame(assay(rld),row.names = FALSE)%>%pivot_longer(1:2,names_to="Sample",values_to = "rlog_trans")%>%
ggplot(aes(x=Sample,y=rlog_trans,fill=Sample))+
geom_boxplot(alpha=0.3)+
theme_bw()+
labs(x="Samples",y="rlog transform values")+
theme(legend.title = element_blank(),plot.title = element_text(size=20, face="bold"))

In [ ]:
# Data Prepration for heatmap
merge = merge(res,as.data.frame(assay(rld)),by = "row.names")
merge = merge[order(merge$padj),]
merge = merge%>%dplyr::select("Row.names","Epilepsy","Normal")
merge = as.data.frame(merge,row.names = merge$Row.names)%>%dplyr::select(-Row.names)

In [ ]:
# Top 10 DEGs
pheatmap(head(merge,10),
         treeheight_col=FALSE,
         main = "TOP 10 DEGs",
         angle_col=0,
         border_color=FALSE,
         fontsize_row = 10,
         fontsize_col = 15,
         cellheight = 25,
         colorRampPalette(c("green", "black", "red"))(50))

In [ ]:
# Top 100 DEGs
pheatmap(head(merge,100),
         treeheight_col=FALSE,
         main = "TOP 100 DEGs",
         angle_col=0,
         border_color=FALSE,
         fontsize_row = 5,
         fontsize_col = 15,
         gaps_col = c(2),
         cellheight = 3,
         cellwidth = 150,
         show_rownames = FALSE,
         colorRampPalette(c("green", "black", "red"))(50))